In [3]:
import requests
from bs4 import BeautifulSoup

In [9]:
cookies = {
    'PHPSESSID': 'h2glnidvr89u8th4p3qigkb454',
    '_ga': 'GA1.1.1495483368.1700470376',
    '_ga_NT9PXBDS5B': 'GS1.1.1700492954.2.1.1700492971.43.0.0',
}

headers = {
    'authority': 'www.newera.edu.my',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-MY,en;q=0.9,en-US;q=0.8,ms;q=0.7',
    'cache-control': 'no-cache',
    # 'cookie': 'PHPSESSID=h2glnidvr89u8th4p3qigkb454; _ga=GA1.1.1495483368.1700470376; _ga_NT9PXBDS5B=GS1.1.1700492954.2.1.1700492971.43.0.0',
    'pragma': 'no-cache',
    'sec-ch-ua': '"Google Chrome";v="119", "Chromium";v="119", "Not?A_Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'none',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
}

In [21]:
r = requests.get('https://www.newera.edu.my/publication.php?id=4805&pub=mjcs', cookies = cookies, headers = headers)

In [22]:
soup = BeautifulSoup(r.content)

In [23]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if '&pub=mjcs' in a_.get('href', '') and 'http' not in a_.get('href', '')]
a

['publication.php?id=4803&pub=mjcs',
 'publication.php?id=4887&pub=mjcs',
 'publication.php?id=4805&pub=mjcs',
 'publication.php?id=4804&pub=mjcs',
 'publication.php?id=4807&pub=mjcs',
 'publication.php?id=4808&pub=mjcs',
 'publication.php?id=4831&pub=mjcs',
 'publication.php?id=4832&pub=mjcs',
 'publication.php?id=4830&pub=mjcs',
 'publication.php?id=4835&pub=mjcs',
 'publication.php?id=4836&pub=mjcs',
 'publication.php?id=4827&pub=mjcs',
 'publication.php?id=4828&pub=mjcs',
 'publication.php?id=4825&pub=mjcs',
 'publication.php?id=4826&pub=mjcs',
 'publication.php?id=4823&pub=mjcs',
 'publication.php?id=4824&pub=mjcs',
 'publication.php?id=4822&pub=mjcs']

In [25]:
from tqdm import tqdm

pdfs = []

for a_ in tqdm(a):
    url = f'http://www.newera.edu.my/{a_}'
    r = requests.get(url, cookies = cookies, headers = headers)
    soup = BeautifulSoup(r.content)
    a_ = [a_.get('href').replace('/article/view/', '/article/download/') for a_ in soup.find_all('a') if '/article/view/' in a_.get('href', '') or a_.get('href', '').endswith('.pdf')]
    pdfs.extend(a_)
    
len(set(pdfs))

100%|███████████████████████████████████████████| 18/18 [00:03<00:00,  4.76it/s]


87

In [27]:
pdfs = sorted(list(set(pdfs)))

In [34]:
import os

for i in tqdm(range(len(pdfs))):
    filename = os.path.join('newera.edu.my', f'{i}.pdf')
    if os.path.exists(filename):
        continue
        
    u = pdfs[i].split('/files')[1]
    u = f'https://www.newera.edu.my/files{u}'
        
    r = requests.get(u, cookies = cookies, headers = headers)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|███████████████████████████████████████████| 87/87 [01:12<00:00,  1.20it/s]


In [35]:
from glob import glob
from tika import parser
import json

TIKA_HOST = 'http://localhost:9998'

In [ ]:
files = glob('newera.edu.my/*.pdf')

with open('newera.edu.my.jsonl', 'w') as fopen:
    for f in tqdm(files):
        raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
        body = BeautifulSoup(raw_xml['content']).find('body')
        t = '\n'.join(body.findAll(text=True))
        fopen.write(f'{json.dumps(t)}\n')

 25%|██████████▊                                | 22/87 [00:21<01:02,  1.03it/s]